In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('C:/Users/aravi/Downloads/kaggle_competions/kaggle_competions_fake_news/train.csv')

In [3]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
train.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
train = train.dropna()

In [6]:
train.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
train.shape

(18285, 5)

In [8]:
X = train.drop('label',axis = 1)
y = train['label']

In [9]:
# let's start the removing the stopwords and do lematization

In [10]:
messages = X.copy()

In [11]:
messages.reset_index(inplace = True) # it help to take the for loop properly

In [12]:
import nltk
from nltk.corpus import stopwords
import re


from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
corpus = []

In [13]:
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ lemmatizer.lemmatize(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional;

C:\Users\aravi\anaconda3\envs\tensflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\aravi\anaconda3\envs\tensflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\aravi\anaconda3\envs\tensflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\aravi\anaconda3\envs\tensflow\lib\site-packages\tensorf

In [15]:
voc_size = 5000
one_hot_repr = [ one_hot(wor, voc_size) for wor in corpus]

In [16]:
sent_length = 20

embeded_doce = pad_sequences(one_hot_repr, padding = 'pre', maxlen = sent_length)

In [17]:
features = 40
model = Sequential()
model.add(Embedding(voc_size, features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
features = 40
model1 = Sequential()
model1.add(Embedding(voc_size, features, input_length = sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(units = 1, activation = 'sigmoid'))
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [21]:
import numpy as np
X.final = np.array(embeded_doce)
y.final = np.array(y)

C:\Users\aravi\anaconda3\envs\tensflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [22]:
X.final.shape
#y.final.shape

(18285, 20)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.final, y.final, test_size=0.33, random_state=42)

In [24]:
model1.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 1,batch_size = 64)

Train on 12250 samples, validate on 6035 samples
12250/12250 [==============================] - 14s 1ms/sample - loss: 0.3181 - acc: 0.8482 - val_loss: 0.2078 - val_acc: 0.9140


In [25]:
y_pred = model.predict_classes(X_test)

In [33]:
y_pred1 = model1.predict_classes(X_test)

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [35]:
print(confusion_matrix(y_test, y_pred1))
print('this is for model1 BRNN')

[[3090  329]
 [ 190 2426]]
this is for model1 BRNN


In [36]:
print(confusion_matrix(y_test, y_pred))
print('this is for model LSTM')

[[2189 1230]
 [1331 1285]]
this is for model LSTM


In [37]:
print(accuracy_score(y_test, y_pred1))
print('this is for model1 BRNN')

0.9140016570008285
this is for model1 BRNN


In [38]:
print(accuracy_score(y_test, y_pred))
print('this is for model LSTM')

0.5756420878210439
this is for model LSTM


In [32]:
# BRNN works too well on first epoch i have low spech that's why one epoch
# if you have good one do more than 60 epoch that will good.